# Procesamiento HTS
## Fases de Tratado
### Fase 0: Configuración General

El objetivo de este script es procesar la base de datos de tarifas de Estados Unidos (HTS - Harmonized Tariff Schedule) y enriquecerla utilizando la base de datos limpia de la LIGIE de México para generar descripciones en español y estructuras jerárquicas comparables.

Instalación de dependencias:
- `pip install pandas xlsxwriter openpyxl`

Librerías:
- **pandas (pd):** Manipulación de estructuras de datos (DataFrames).
- **xlsxwriter:** Motor de escritura para Excel con formato avanzado (celdas ajustadas, negritas, bordes).
- **os/sys:** Manejo de rutas del sistema operativo y control de flujo.

Variables Globales:
- **Rutas:** Se definen inicialmente para facilitar el cambio entre entornos de desarrollo.

In [10]:
# pip install pandas xlsxwriter openpyxl

import pandas as pd
import xlsxwriter
import os
import sys

PATH_LIGIE = r"../data/intermediate/LIGIE_Limpia.xlsx"
PATH_HTS_INPUT = r"../data/raw/htsdata.csv"
PATH_OUTPUT = r"../data/intermediate/HTS_Rellenada.xlsx"

print("--- CONFIGURACIÓN CARGADA ---")
print(f"Input LIGIE: {PATH_LIGIE}")
print(f"Input HTS:   {PATH_HTS_INPUT}")
print(f"Output:      {PATH_OUTPUT}")

--- CONFIGURACIÓN CARGADA ---
Input LIGIE: ../data/intermediate/LIGIE_Limpia.xlsx
Input HTS:   ../data/raw/htsdata.csv
Output:      ../data/intermediate/HTS_Rellenada.xlsx


### Fase 0.5: Definición de Funciones

En esta sección definimos la lógica modular del proceso. Se sigue la nomenclatura estricta para identificar la naturaleza de cada función:

**1. Funciones Auxiliares (`_nombre`)**
Herramientas de soporte para tareas repetitivas de limpieza, validación y formateo de cadenas.

- **`_forzar_dos_digitos` / `_forzar_un_digito`**: Normalización de strings. Vital para asegurar que las llaves de búsqueda (ej. "01" vs "1") coincidan entre bases de datos.
- **`_limpiar_y_normalizar_hts`**: Estandariza el input americano, eliminando basura, quitando puntos y asegurando una longitud par (8 dígitos).
- **`_formatear_hts_con_puntos`**: Formateador visual. Convierte el código limpio (`01012101`) en formato legible con puntos (`0101.21.01`) para el reporte final.
- **`_concatenar_descripciones`**: Constructor de texto. Genera la "historia completa" de una fracción, uniendo jerárquicamente el nombre de su Sección, Capítulo, Partida, Subpartida y Fracción.

**2. Funciones Críticas (`__nombre__`)**
Lógica estructural compleja o delicada que gestiona la salida de datos.

- **`__escribir_hoja__`**: Motor de exportación Excel. Controla la librería `xlsxwriter` para crear pestañas, aplicar negritas y bordes. Su función más importante es calcular el ajuste de texto (Wrap Text) y los anchos de columna para que las descripciones largas sean legibles.

**3. Funciones Principales (`nombre`)**
Orquestadores que ejecutan procesos completos y gestionan el flujo de datos.

- **`cargar_diccionarios_ligie`**: Integrador de Referencias. Lee el archivo maestro de la LIGIE (específicamente el Panel Extendido) y construye los diccionarios de mapeo necesarios para traducir los códigos numéricos a descripciones jerárquicas (Sección, Capítulo, Partida).
- **`limpiar_hts_inicial`**: Cargador y Filtro Inicial. Importa la base de datos cruda del HTS, selecciona las columnas de interés (Código, Descripción, Tasas) y elimina registros administrativos o temporales (como el Capítulo 99) que no son relevantes para el análisis comercial.

In [ ]:
# --- FUNCIONES AUXILIARES (Limpieza y Formato) ---

def _forzar_dos_digitos(serie):
    """Convierte números a strings de 2 dígitos (ej. '1' -> '01')."""
    return serie.fillna('').astype(str).str.replace(r'\.0$', '', regex=True).str.strip().str.zfill(2)


def _forzar_un_digito(serie):
    """Convierte números a strings de 1 dígito (ej. '1.0' -> '1')."""
    return serie.fillna('').astype(str).str.replace(r'\.0$', '', regex=True).str.strip().str.zfill(1)


def _limpiar_y_normalizar_hts(code):
    """Limpia el código HTS, elimina puntos y ajusta longitud a 8 dígitos pares."""
    c = str(code).replace('.', '').strip()
    if len(c) % 2 != 0: c = '0' + c
    if len(c) >= 10: c = c[:8]
    return c


def _formatear_hts_con_puntos(code_clean):
    """Reinserta los puntos para formato visual (ej. '01012101' -> '0101.21.01')."""
    if len(code_clean) != 8:
        return code_clean 
    return f"{code_clean[:4]}.{code_clean[4:6]}.{code_clean[6:8]}"


def _concatenar_descripciones(row):
    """Une jerárquicamente las descripciones (Sección -> Capítulo -> ... -> Fracción)."""
    partes = [
        row.get('Nombre Sección', ''),
        row.get('Nombre Capítulo', ''),
        row.get('Nombre Partida', ''),
        row.get('Nombre Desdoblamiento', ''),
        row.get('Nombre Subpartida', ''),
        row.get('Descripción Fracción', '')
    ]
    validas = [str(x).strip() for x in partes if str(x).strip() not in ['', 'ND', 'nan']]
    return " ".join(validas)


# --- FUNCIONES PRINCIPALES (Carga y Negocio) ---

def cargar_diccionarios_ligie(path):
    """Lee el Excel Maestro LIGIE y genera diccionarios de mapeo para Secciones y Capítulos."""
    if not os.path.exists(path):
        sys.exit(f"ERROR: No existe {path}")

    print(">> Cargando referencias LIGIE y mapeando Secciones...")
    try:
        # Leemos el Panel Extendido que ya contiene la info procesada
        df = pd.read_excel(path, sheet_name="Panel Extendido", dtype=str)
        
        # Normalización de llaves usando auxiliares
        df['Ref_Cap'] = _forzar_dos_digitos(df['Capítulo'])      
        df['Ref_Part_Suffix'] = _forzar_dos_digitos(df['Partida']) 
        df['Ref_Desdob'] = _forzar_un_digito(df['Desdob.'])      
        df['Ref_Sub'] = _forzar_un_digito(df['Subpartida'])      

        # Construcción de Llaves Compuestas
        df['Key_Cap'] = df['Ref_Cap']
        df['Key_Part'] = df['Ref_Cap'] + df['Ref_Part_Suffix']
        df['Key_Sub'] = df['Key_Part'] + "." + df['Ref_Desdob'] + df['Ref_Sub']
        
        # Sub-dataframe único por capítulo para mapear secciones
        df_caps = df[['Ref_Cap', 'Sección', 'Nombre Sección']].drop_duplicates(subset=['Ref_Cap'])
        
        # Generación de Diccionarios
        diccionarios = {
            'sec_rom': pd.Series(df_caps['Sección'].values, index=df_caps['Ref_Cap']).to_dict(),
            'sec_nom': pd.Series(df_caps['Nombre Sección'].values, index=df_caps['Ref_Cap']).to_dict(),
            'cap': pd.Series(df['Nombre Capítulo'].values, index=df['Key_Cap']).to_dict(),
            'part': pd.Series(df['Nombre Partida'].values, index=df['Key_Part']).to_dict(),
            'subpart': pd.Series(df['Nombre Subpartida'].values, index=df['Key_Sub']).to_dict(),
            'desdob': pd.Series(df['Nombre Desdoblamiento'].values, index=df['Key_Sub']).to_dict()
        }
        print("   Diccionarios listos.")
        return diccionarios

    except Exception as e:
        sys.exit(f"ERROR leyendo LIGIE: {e}")


def limpiar_hts_inicial(path):
    """Carga el archivo raw de HTS, filtra encabezados y columnas innecesarias."""
    if not os.path.exists(path):
        sys.exit(f"ERROR: No existe {path}")
    
    print(">> Procesando HTS Raw...")
    try:
        if path.endswith('.csv'):
            df = pd.read_csv(path, dtype=str)
        else:
            df = pd.read_excel(path, dtype=str)
    except:
        sys.exit("No se pudo leer el archivo HTS.")

    cols_interes = ["HTS Number", "Description", "Unit of Quantity", "General Rate of Duty", "Special Rate of Duty"]
    df = df[cols_interes].copy().fillna('')
    
    # Limpieza de Capítulo 99 (Disposiciones especiales temporales)
    df['HTS_Temp'] = df['HTS Number'].str.replace('.', '', regex=False).str.strip()
    mask_99 = df['HTS_Temp'].str.startswith('99', na=False)
    if mask_99.any():
        idx = df[mask_99].index[0]
        df = df.iloc[:idx].copy()
    
    return df.drop(columns=['HTS_Temp'])


# --- FUNCIONES CRÍTICAS (Infraestructura) ---

def __escribir_hoja__(dataframe, nombre_hoja, writer, workbook):
    """
    Motor de escritura Excel. Maneja formatos de encabezado, anchos de columna
    y ajuste de texto (Wrap Text) para descripciones largas.
    """
    # Definición de formatos
    fmt_header = workbook.add_format({
        'bold': True, 'border': 1, 'bg_color': '#BDD7EE',
        'valign': 'vcenter', 'align': 'center', 'font_name': 'Arial', 'font_size': 10, 'text_wrap': True
    })
    fmt_body = workbook.add_format({'border': 1, 'valign': 'top', 'font_name': 'Arial', 'font_size': 9})
    fmt_wrap = workbook.add_format({'border': 1, 'valign': 'top', 'font_name': 'Arial', 'font_size': 9, 'text_wrap': True})

    # Escritura de datos crudos (sin header aún)
    dataframe.to_excel(writer, sheet_name=nombre_hoja, index=False, startrow=1, header=False)
    worksheet = writer.sheets[nombre_hoja]
    
    # Escritura de encabezados con formato
    headers = dataframe.columns.tolist()
    for col_idx, header in enumerate(headers):
        worksheet.write(0, col_idx, header, fmt_header)
        
    # Aplicación de estilos columna por columna
    ancho_texto_largo = 37 if ("Textual" in nombre_hoja or "Extendido" in nombre_hoja) else 50

    for col_idx, col_name in enumerate(headers):
        columnas_wrap = ["Description", "Descripción", "Nombre", "Completa", "Special Rate"]
        es_largo = any(x in col_name for x in columnas_wrap)
        
        # Formato de celdas del cuerpo (aplicado a toda la columna)
        estilo = fmt_wrap if es_largo else fmt_body
        
        for row_idx in range(len(dataframe)):
            val = dataframe.iloc[row_idx, col_idx]
            if pd.isna(val): val = ""
            worksheet.write(row_idx + 1, col_idx, val, estilo)
        
        # Ajuste de Anchos
        if es_largo:
            worksheet.set_column(col_idx, col_idx, ancho_texto_largo)
        elif "General Rate" in col_name:
            worksheet.set_column(col_idx, col_idx, 15)
        elif len(col_name) <= 10 and "Unit" not in col_name:
            worksheet.set_column(col_idx, col_idx, 10)
        else:
            worksheet.set_column(col_idx, col_idx, 12)

### Fase 1: Carga de Referencias
Se cargan los diccionarios generados previamente en el proceso LIGIE. Esto permite "traducir" o enriquecer los códigos HTS (que solo traen descripción en inglés de la fracción) con la estructura jerárquica en español (Sección, Capítulo, Partida).

In [12]:
DICCIONARIOS_LIGIE = cargar_diccionarios_ligie(PATH_LIGIE)

>> Cargando referencias LIGIE y mapeando Secciones...
   Diccionarios listos.


### Fase 2: Limpieza y Normalización
Se carga la base HTS cruda y se aplican las funciones auxiliares de normalización para preparar los códigos para el cruce de datos.

In [13]:
DF_HTS_RAW = limpiar_hts_inicial(PATH_HTS_INPUT)

print(">> Normalizando códigos HTS...")
# Filtramos solo filas con tasas definidas para evitar encabezados vacíos
df_panels = DF_HTS_RAW[DF_HTS_RAW['General Rate of Duty'] != ''].copy()

# Aplicación de limpieza y formateo visual
df_panels['HTS_8_Clean'] = df_panels['HTS Number'].apply(_limpiar_y_normalizar_hts)
df_panels['HTS_Dotted'] = df_panels['HTS_8_Clean'].apply(_formatear_hts_con_puntos)

print(">> Proceso completado.")

>> Procesando HTS Raw...
>> Normalizando códigos HTS...
>> Proceso completado.


### Fase 3: Construcción de Paneles
Se generan las diferentes vistas (DataFrames) que se exportarán. Se utiliza la lógica de mapeo (`map`) contra los diccionarios cargados en la Fase 1.

- **Panel Numérico:** Enfoque en códigos desglosados.
- **Panel Textual:** Enfoque en descripciones jerárquicas.
- **Panel Extendido:** Matriz completa.
- **Concatenado:** Descripción unificada.

In [14]:
clean = df_panels['HTS_8_Clean'] 
dotted = df_panels['HTS_Dotted'] 

# Generación de llaves para búsqueda en diccionarios
key_cap = clean.str[:2]
key_part = clean.str[:4]
key_sub = clean.str[:4] + "." + clean.str[4:6]

# --- A. PANEL NUMÉRICO ---
df_num = pd.DataFrame()
df_num['Código']   = dotted
df_num['Sección']  = key_cap.map(DICCIONARIOS_LIGIE['sec_rom']).fillna('') 
df_num['Capítulo'] = clean.str[:2]
df_num['Partida']  = clean.str[2:4]
df_num['Desdoblamiento'] = clean.str[4:5]
df_num['Subpartida']     = clean.str[5:6]
df_num['Fracción']       = clean.str[6:8]
df_num['Unit of Quantity'] = df_panels['Unit of Quantity']
df_num['General Rate of Duty'] = df_panels['General Rate of Duty']

# --- B. PANEL TEXTUAL ---
df_txt = pd.DataFrame()
df_txt['Código'] = dotted
df_txt['Nombre Sección'] = key_cap.map(DICCIONARIOS_LIGIE['sec_nom']).fillna('')
df_txt['Nombre Capítulo'] = key_cap.map(DICCIONARIOS_LIGIE['cap']).fillna('')
df_txt['Nombre Partida']  = key_part.map(DICCIONARIOS_LIGIE['part']).fillna('')
df_txt['Nombre Desdoblamiento'] = key_sub.map(DICCIONARIOS_LIGIE['desdob']).fillna('')
df_txt['Nombre Subpartida']     = key_sub.map(DICCIONARIOS_LIGIE['subpart']).fillna('')
df_txt['Descripción Fracción'] = df_panels['Description']
df_txt['Unit of Quantity'] = df_panels['Unit of Quantity']
df_txt['General Rate of Duty'] = df_panels['General Rate of Duty']

# --- C. PANEL EXTENDIDO ---
df_ext = pd.DataFrame()
df_ext['Código'] = dotted
df_ext['Sección'] = df_num['Sección']
df_ext['Nombre Sección'] = df_txt['Nombre Sección']
df_ext['Capítulo']        = df_num['Capítulo']
df_ext['Nombre Capítulo'] = df_txt['Nombre Capítulo']
df_ext['Partida']        = df_num['Partida']
df_ext['Nombre Partida'] = df_txt['Nombre Partida']
df_ext['Desdoblamiento'] = df_num['Desdoblamiento']
df_ext['Nombre Desdoblamiento'] = df_txt['Nombre Desdoblamiento']
df_ext['Subpartida']     = df_num['Subpartida']
df_ext['Nombre Subpartida'] = df_txt['Nombre Subpartida']
df_ext['Fracción']             = df_num['Fracción']
df_ext['Descripción Fracción'] = df_txt['Descripción Fracción']
df_ext['Unit of Quantity']     = df_panels['Unit of Quantity']
df_ext['General Rate of Duty'] = df_panels['General Rate of Duty']

# --- D. PANEL CONCATENADO ---
df_concat = pd.DataFrame()
df_concat['Código'] = dotted
df_concat['Descripción Completa'] = df_txt.apply(_concatenar_descripciones, axis=1)
df_concat['Unit of Quantity']     = df_panels['Unit of Quantity']
df_concat['General Rate of Duty'] = df_panels['General Rate of Duty']

### Fase 4: Exportación Final
Utilizando el motor `xlsxwriter` y la función crítica `__escribir_hoja__`, se genera el archivo Excel final con múltiples pestañas y formato profesional.

In [15]:
print(f">> Generando archivo Excel: {PATH_OUTPUT}")

try:
    writer = pd.ExcelWriter(PATH_OUTPUT, engine='xlsxwriter')
    workbook = writer.book

    # Llamadas a función crítica de exportación
    __escribir_hoja__(DF_HTS_RAW, "HTS Cleaned", writer, workbook)
    __escribir_hoja__(df_num, "Panel Numérico", writer, workbook)
    __escribir_hoja__(df_txt, "Panel Textual", writer, workbook)
    __escribir_hoja__(df_ext, "Panel Extendido", writer, workbook)
    __escribir_hoja__(df_concat, "Concatenado", writer, workbook)

    writer.close()
    print("¡PROCESO FINALIZADO EXITOSAMENTE!")

except Exception as e:
    print(f"ERROR CRÍTICO EN EXPORTACIÓN: {e}")

>> Generando archivo Excel: ../data/intermediate/HTS_Rellenada.xlsx
¡PROCESO FINALIZADO EXITOSAMENTE!
